In [1]:
import pandas as pd

# 데이터 로드, 정제

In [2]:
# 데이터는 일단 비공개 
df = pd.read_csv('./ynnews/article-titles.csv', names=['id','channel','title','is_title','is_content','is_human'])

In [3]:
df[:4]

,id,channel,title,is_title,is_content,is_human
0,252,연합뉴스,[영상] '효리네 민박'의 엄청난 파급효과…내국인 100만명 이끌어,NaN,NaN,NaN
1,253,연합뉴스,"""주택 공시가격 30% 오르면 지역가입자 건보료 평균 4% 인상""",NaN,NaN,NaN
2,254,연합뉴스,"파업 국민銀 일부지점서 고객 '헛걸음'…""거점점포는 정상영업""(종합)",NaN,NaN,NaN
3,255,연합뉴스,"체육회, 15일 이사회서 사무총장·선수촌장 인선 발표",NaN,NaN,NaN


In [4]:
#타이틀 없는 것들이 있음. 제거
df = df[~df['title'].isna()]

# 데이터 살펴보기

In [5]:
df.is_human.value_counts()

False    3524
True     1320
Name: is_human, dtype: int64

인간의 판단이 개입된 기사가 약 4800건.  
개중 인간이 문제기사로 판단한 기사가 1320건.  
비율을 맞추기 위해 비문제기사도 1300건만 뽑는다.  

In [6]:
# 일단 판단이 있는 기사들만 모음
df_data = df[~df.is_human.isna()]

In [7]:
# 나중엔 다르게 할것임
# 일단 귀찮고 드럽고 로직이 보이는 코드로 써봄
df_data_t = df_data[df_data.is_human==True]
df_data_f = df_data[df_data.is_human==False]
df_data_f = df_data_f.sample(frac=0.375)
df_data = pd.concat([df_data_t, df_data_f])

In [8]:
# 비율확인
df_data.is_human.value_counts()

False    1322
True     1320
Name: is_human, dtype: int64

In [9]:
# 기존 True/False로 되어있는 것을 1:0으로 바꿈. 1이 문제기사
df_data.is_human = df_data.is_human.replace({True:1, False:0})

In [10]:
# 그냥 concat했으니까 앞쪽이 다 문제기사임. 섞어줌.
df_data = df_data.sample(frac=1)

# train과 test를 2000:600 으로 쪼갬. 비율은 달라도 됨.
df_train = df_data[:2000]
df_test = df_data[2000:]

## 여기서부터 머신러닝

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB 

In [12]:
# 모르는 것이 많을텐데 설명 할 수가 없음. sklearn 메뉴얼을 보십시요
# 여기가 모델학습 부분

# analyzer를 word혹은 char/char_wb로 할 수 있음
# ngram도 적절히 섞어주면 좋음
vect = TfidfVectorizer(analyzer='word', ngram_range=(1,3))
# 원래 벡터라이즈(임베딩) 하기 전에 형태소분석기로 토큰을 더 잘 쪼개주면 좋음.
# konlpy 안 깔려있어서 그냥 함ㅋㅋㅋㅋ

# 벡터라이저를 학습시키고(fit) 텍스트->벡터변환(transform)까지 하는 함수
X_train = vect.fit_transform(df_train.title)

# Multinomial NB =  다항 naive bayes인데 다른거 쓰면 성능이 더 좋을것임(아마)
classifier = MultinomialNB()
# 분류기는 학습fit만 하면 됨.
classifier.fit(X_train, df_train.is_human)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [13]:
# 여기가 모델을 통한 예측
X_test = vect.transform(df_test.title)
df_test = df_test.assign(predicted=classifier.predict(X_test))

In [14]:
# 모델이 틀리게 예측한것만 모아보기
df_test[df_test.predicted != df_test.is_human]

,id,channel,title,is_title,is_content,is_human,predicted
72458,203392,연합뉴스,"""임신 여군 단축근무하기 힘들다""…여군들 국방장관에 하소연",군인 성별 표시,군인 성별 표시,0,1
236559,543184,연합뉴스,[2보] 강선영 준장 여군 첫 소장 진급…장군 인사,군인 성별 표시,군인 성별 표시,0,1
103307,266434,연합뉴스,소년체전 응원 온 청주 고교 코치가 중학생 선수 폭행,소년 소녀 표시,NaN,0,1
35413,80259,연합뉴스,자폐증 10대 아들과 아버지 모텔서 안타까운 죽음…극단적 선택,자녀 성별 표시,자녀 성별 표시,1,0
235342,540710,연합뉴스,"인니 여성, 출산한 아기를 세탁기 속에…""부끄러워서""",성별 표시,성별 표시,0,1
...,...,...,...,...,...,...,...
111807,283914,연합뉴스,[영상] 버스 지붕에 올라탄 소년들…터널 들어갔다가 그만,소년 소녀 표시,소년 소녀 표시,0,1
63411,184558,연합뉴스,"'왓칭' 강예원 ""강인하고 주체적인 여성 만들고 싶었다""",성별 표시,성별 표시,0,1
148886,361444,연합뉴스,"셋째 부인 얻으려 남편 살해한 印 '외식왕', 형기 시작하자 사망",부부 성별 표시,부부 성별 표시,1,0
233652,537212,연합뉴스,'성희롱 의혹' 김동열 중기연구원장 해임(종합),NaN,직원 성별 표기,0,1


In [15]:
sum(df_test.is_human == df_test.predicted)

511

## 전체데이터로 보기 (인간이 판단한 적 없는 데이터)

In [16]:
proba = classifier.predict_proba(vect.transform(df.title))

In [17]:
df = df.assign(pred=[p[1] for p in proba])

In [18]:
df_nan = df[df.is_human.isna()]

In [19]:
df_nan[df_nan.title.str.contains('\d0대')].sort_values(by='pred')

,id,channel,title,is_title,is_content,is_human,pred
183961,434358,연합뉴스,'조국 사태'에 20∼40대女 정부 지지 균열?…맘카페도 '시끌',NaN,부부 성별 표시,NaN,0.191816
141940,346961,연합뉴스,횡성 실종 70대 낚시꾼 저수지서 숨진 채 발견,NaN,NaN,NaN,0.205558
247509,565259,연합뉴스,"와이즈와이어즈, '2019 대한민국 일하기 좋은 100대 기업' 선정",NaN,NaN,NaN,0.219410
61573,180154,연합뉴스,"""나물 캐려다가""…울릉도서 실종 70대 사흘 만에 숨진 채 발견",NaN,NaN,NaN,0.230304
96063,251230,연합뉴스,완도 조약도 해상서 실종 70대 숨진 채로 발견,NaN,NaN,NaN,0.236794
...,...,...,...,...,...,...,...
23070,26036,연합뉴스,부부싸움 하다 흉기로 아내 살해한 50대 체포(종합),부부 성별 표시,부부 성별 표시,NaN,0.921810
122769,307063,연합뉴스,동료 추행해 숨지게 한 40대 '항소 기각'…원심대로 징역 6년,NaN,자녀 성별 표시,NaN,0.923166
12953,13450,연합뉴스,전통무예 가르친다며 수련생 때려 숨지게 한 50대 관장 구속,NaN,부부 성별 표시,NaN,0.924032
106470,273015,연합뉴스,"목포해경, 동료 흉기로 찌른 20대 베트남 선원 구속",NaN,NaN,NaN,0.926625


In [20]:
df_nan.sort_values(by='pred',ascending=False)[:60]['title'].values

array(['인천 서구청장 성추행 의혹…공무원 노조 "수사의뢰 결정"',
       '술자리서 10년지기 때려 숨지게 한 30대 징역 3년 선고',
       '목포해경, 동료 흉기로 찌른 20대 베트남 선원 구속',
       '전통무예 가르친다며 수련생 때려 숨지게 한 50대 관장 구속',
       "동료 추행해 숨지게 한 40대 '항소 기각'…원심대로 징역 6년",
       '부부싸움 하다 흉기로 아내 살해한 50대 체포(종합)', '갈등 빚던 노모 때려 숨지게 한 40대 패륜아들 검거',
       '"왜 인사 안 해" 후배 때려 숨지게 한 30대 항소심도 징역 4년',
       "'오진 구속' 의사 3명 항소심서 무죄 선고·집행유예 감형", '지인과 술값 문제로 다투다 흉기 휘두른 50대 검거',
       '전통무예 가르친다며 수련생 때려 숨지게 한 50대 관장 구속(종합)',
       '옛 연인 흉기로 찌른 50대 항소심도 징역 3년 6개월',
       "'여자친구 성추행해서'…친구 때려 숨지게 한 20대 검거", '숙박료 갈등…모텔 여주인 때려 숨지게 한 40대 구속',
       '이별 통보한 전 여친 흉기로 마구 찌른 50대…대치 끝 검거',
       '"왜 반말해" 함께 술 마시던 남성 때려 숨지게 한 50대 구속',
       '층간소음 시비로 위층 주민에 흉기 휘두른 50대 징역 3년',
       '병문안 가서 지인 10대 딸 성추행한 30대 징역 2년', '말다툼하다 직장동료 흉기 휘둘러 살해한 50대 검거',
       '곡성 농장주 살해·유기 40대 항소심도 징역 35년', '공항철도 열차서 성추행한 40대 검거…20대 피해자가 신고',
       '층간소음에 흉기 휘두른 50대 2심도 징역 4년', '산책하던 80대 노인 때려 숨지게 한 20대 징역 15년',
       "'돈 왜 안 갚아'…지인에게 흉기 휘두른 60대 체포",
       '"왜 욕해" 취객 때려 숨지게 한 40대, 

# 기타

In [21]:
df[df['title'].str.contains('"') & df['title'].str.contains('왜')]

,id,channel,title,is_title,is_content,is_human,pred
795,1047,연합뉴스,"""왜 이런 일이…"" 김천서 탱크폭발로 20대 근로자 안타까운 죽음",NaN,NaN,NaN,0.547003
1128,1380,연합뉴스,"""왜 저를 낳았나요"" 부모를 고소한 소년…영화 '가버나움'",소년 소녀 표시,자녀 성별 표시,NaN,0.356524
3119,3371,연합뉴스,"CNN ""연예지가 왜 베이조스 불륜설 추적했을까…트럼프 커넥션?""",NaN,부부 성별 표시,NaN,0.345230
3160,3412,연합뉴스,"[함보세] ""지가 뭔데?"" 나경원 공개 비난한 지만원, 왜?",NaN,NaN,NaN,0.354748
4163,4415,연합뉴스,"""식당 앞에서 왜 떠들어""…종업원에게 뺨 맞은 행인 의식불명",NaN,NaN,NaN,0.514819
...,...,...,...,...,...,...,...
254047,579090,연합뉴스,"'민식이' 부모, 나경원에 ""왜 아이들을 협상카드로 쓰냐"" 격분",NaN,부모님 성별 표시,NaN,0.405443
254170,579377,연합뉴스,"사이버외교사절단 美여대생 ""왜곡 한국사 바로잡고 싶어요""",NaN,학생 성별 표기,True,0.691120
254894,580817,연합뉴스,"[D스토리] ""엄마는 왜 치마 안 입어?""",NaN,NaN,NaN,0.455884
255612,582193,연합뉴스,"""14살 내게 왜""…이라크여성, IS 성폭행범에 절규 끝 혼절",NaN,자녀 성별 표시,NaN,0.443770


In [22]:
df[df['title'].str.contains('홧김')]

,id,channel,title,is_title,is_content,is_human,pred
4138,4390,연합뉴스,"'가족끼리 어떻게…' 홧김에·술 취해서, 방화 사건 잇따라",NaN,자녀 성별 표시,True,0.740223
19511,20008,연합뉴스,'남친 이별 통보에 홧김 방화' 10대 여성 두 달여 만에 구속,성별 표시,성별 표시,True,0.805344
22070,22567,연합뉴스,"홧김에 편의점 불 질러 점주 사망…2심, 형량 올려 징역 17년",NaN,부부 성별 표시,NaN,0.751109
22707,24740,연합뉴스,홧김에 고시텔에 불 지른 60대 2심도 징역 4년,NaN,NaN,NaN,0.848535
24384,32460,연합뉴스,"""왜 돈 안 갚아"" 홧김에 친구 집에서 명품 가방 등 훔쳐",NaN,NaN,NaN,0.736790
27538,46335,연합뉴스,"홧김에 반려견 던진 수분양인 ""받을 줄 알았는데…"" 때늦은 후회",NaN,NaN,NaN,0.681134
65722,189515,연합뉴스,"""홧김에…"" 헤어진 연인 운영하는 모텔에 불 지른 50대",NaN,NaN,NaN,0.723382
65955,189894,연합뉴스,"경찰 ""황하나 '경찰청장 베프' 발언 사실 아냐…홧김에 나온 것""",NaN,NaN,NaN,0.662288
69088,196521,연합뉴스,"아파트 방화살인범 ""사회적으로 계속 불이익당해…홧김에 방화""(종합)",NaN,NaN,NaN,0.763783
70695,199823,연합뉴스,"홧김에 ""남편이 휘발유 들고 탔다""…부산지하철 4분간 정차",NaN,부부 성별 표시,NaN,0.642427


In [23]:
crime_keywords = ["영장","구속","구형","붙잡아","실형","덜미","징역","검거","벌금","체포","소동"]
keywords_str = '|'.join(crime_keywords)

In [24]:
df[df['title'].str.startswith('"') & df['title'].str.contains("keywords_str")][60:120]

,id,channel,title,is_title,is_content,is_human,pred
